In [46]:
# Milestone 1: DataLoader

# Data Loading

# Import required libraries.
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("DataLoader") \
    .getOrCreate()

In [47]:
# Load dataset from flights.parquet into a PySpark DataFrame
flights_df = spark.read.parquet("/Users/sonushah/Desktop/final_assignments/Flights 1m.parquet")

In [48]:
# Display the schema of the DataFrame to understand its structure
flights_df.printSchema()

root
 |-- FL_DATE: date (nullable = true)
 |-- DEP_DELAY: short (nullable = true)
 |-- ARR_DELAY: short (nullable = true)
 |-- AIR_TIME: short (nullable = true)
 |-- DISTANCE: short (nullable = true)
 |-- DEP_TIME: float (nullable = true)
 |-- ARR_TIME: float (nullable = true)



In [49]:
flights_df.show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2006-01-01|        5|       19|     350|    2475| 9.083333|12.483334|
|2006-01-02|      167|      216|     343|    2475|11.783334|15.766666|
|2006-01-03|       -7|       -2|     344|    2475| 8.883333|12.133333|
|2006-01-04|       -5|      -13|     331|    2475| 8.916667|    11.95|
|2006-01-05|       -3|      -17|     321|    2475|     8.95|11.883333|
|2006-01-06|       -4|      -32|     320|    2475| 8.933333|11.633333|
|2006-01-08|       -3|       -2|     346|    2475|     8.95|12.133333|
|2006-01-09|        3|        0|     334|    2475|     9.05|12.166667|
|2006-01-10|       -7|      -21|     334|    2475| 8.883333|11.816667|
|2006-01-11|        8|      -10|     321|    2475| 9.133333|     12.0|
|2006-01-12|       -5|      -27|     321|    2475| 8.916667|11.716666|
|2006-

In [50]:
# Data Preprocessing

# NULL Value Check
# Assess each column for NULL values.

from pyspark.sql.functions import col, when, split

null_counts = {col_name: flights_df.filter(col(col_name).isNull()).count() for col_name in flights_df.columns}
print("NULL counts per column:", null_counts)


NULL counts per column: {'FL_DATE': 0, 'DEP_DELAY': 0, 'ARR_DELAY': 0, 'AIR_TIME': 0, 'DISTANCE': 0, 'DEP_TIME': 0, 'ARR_TIME': 0}


In [51]:
# Devise strategies for handling columns with significant NULL values (e.g., removal, imputation).

# For demonstration, let's impute NULL values with the mean for numeric columns and with a default
# value for string columns.

for col_name in flights_df.columns:
    if null_counts[col_name] > 0:
        if flights_df.schema[col_name].dataType == "string":
            default_value = "Unknown"  # Specify the default value for string columns
            flights_df = flights_df.fillna(default_value, subset=[col_name])
        else:
            mean_value = flights_df.select(col_name).agg({col_name: "mean"}).collect()[0][0]
            flights_df = flights_df.fillna(mean_value, subset=[col_name])
            

# Show the DataFrame after handling NULL values
flights_df.show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2006-01-01|        5|       19|     350|    2475| 9.083333|12.483334|
|2006-01-02|      167|      216|     343|    2475|11.783334|15.766666|
|2006-01-03|       -7|       -2|     344|    2475| 8.883333|12.133333|
|2006-01-04|       -5|      -13|     331|    2475| 8.916667|    11.95|
|2006-01-05|       -3|      -17|     321|    2475|     8.95|11.883333|
|2006-01-06|       -4|      -32|     320|    2475| 8.933333|11.633333|
|2006-01-08|       -3|       -2|     346|    2475|     8.95|12.133333|
|2006-01-09|        3|        0|     334|    2475|     9.05|12.166667|
|2006-01-10|       -7|      -21|     334|    2475| 8.883333|11.816667|
|2006-01-11|        8|      -10|     321|    2475| 9.133333|     12.0|
|2006-01-12|       -5|      -27|     321|    2475| 8.916667|11.716666|
|2006-

In [52]:
# Remove rows with any null values.
flights_df_no_null = flights_df.na.drop()

In [53]:
# Show the DataFrame after removing null values
flights_df_no_null.show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2006-01-01|        5|       19|     350|    2475| 9.083333|12.483334|
|2006-01-02|      167|      216|     343|    2475|11.783334|15.766666|
|2006-01-03|       -7|       -2|     344|    2475| 8.883333|12.133333|
|2006-01-04|       -5|      -13|     331|    2475| 8.916667|    11.95|
|2006-01-05|       -3|      -17|     321|    2475|     8.95|11.883333|
|2006-01-06|       -4|      -32|     320|    2475| 8.933333|11.633333|
|2006-01-08|       -3|       -2|     346|    2475|     8.95|12.133333|
|2006-01-09|        3|        0|     334|    2475|     9.05|12.166667|
|2006-01-10|       -7|      -21|     334|    2475| 8.883333|11.816667|
|2006-01-11|        8|      -10|     321|    2475| 9.133333|     12.0|
|2006-01-12|       -5|      -27|     321|    2475| 8.916667|11.716666|
|2006-

In [58]:
# Handle records with empty or NULL FL_Date.
flights_df_empty_null = flights_df.filter(col("FL_DATE").isNotNull() | (col("FL_Date") != ""))

In [59]:
flights_df_empty_null.show()

+----------+---------+---------+--------+--------+---------+---------+----------+----+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|      Date|Time|
+----------+---------+---------+--------+--------+---------+---------+----------+----+
|2006-01-01|        5|       19|     350|    2475| 9.083333|12.483334|2006-01-01|null|
|2006-01-02|      167|      216|     343|    2475|11.783334|15.766666|2006-01-02|null|
|2006-01-03|       -7|       -2|     344|    2475| 8.883333|12.133333|2006-01-03|null|
|2006-01-04|       -5|      -13|     331|    2475| 8.916667|    11.95|2006-01-04|null|
|2006-01-05|       -3|      -17|     321|    2475|     8.95|11.883333|2006-01-05|null|
|2006-01-06|       -4|      -32|     320|    2475| 8.933333|11.633333|2006-01-06|null|
|2006-01-08|       -3|       -2|     346|    2475|     8.95|12.133333|2006-01-08|null|
|2006-01-09|        3|        0|     334|    2475|     9.05|12.166667|2006-01-09|null|
|2006-01-10|       -7|      -21|     334|  

In [55]:
# Split FL_Date into Date and Time columns
flights_df = flights_df.withColumn("Date", split(col("FL_Date"), " ")[0])
flights_df = flights_df.withColumn("Time", split(col("FL_Date"), " ")[1])

In [57]:
# Show the DataFrame after FL_Date handling and splitting.
flights_df.show()

+----------+---------+---------+--------+--------+---------+---------+----------+----+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|      Date|Time|
+----------+---------+---------+--------+--------+---------+---------+----------+----+
|2006-01-01|        5|       19|     350|    2475| 9.083333|12.483334|2006-01-01|null|
|2006-01-02|      167|      216|     343|    2475|11.783334|15.766666|2006-01-02|null|
|2006-01-03|       -7|       -2|     344|    2475| 8.883333|12.133333|2006-01-03|null|
|2006-01-04|       -5|      -13|     331|    2475| 8.916667|    11.95|2006-01-04|null|
|2006-01-05|       -3|      -17|     321|    2475|     8.95|11.883333|2006-01-05|null|
|2006-01-06|       -4|      -32|     320|    2475| 8.933333|11.633333|2006-01-06|null|
|2006-01-08|       -3|       -2|     346|    2475|     8.95|12.133333|2006-01-08|null|
|2006-01-09|        3|        0|     334|    2475|     9.05|12.166667|2006-01-09|null|
|2006-01-10|       -7|      -21|     334|  

In [60]:
# Handle outliers(e.g., remove rows with Dep_Delay greater than 100).
flights_df_outliers = flights_df.filter(col("DEP_DELAY") < 100)
flights_df_outliers.show()

+----------+---------+---------+--------+--------+--------+---------+----------+----+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE|DEP_TIME| ARR_TIME|      Date|Time|
+----------+---------+---------+--------+--------+--------+---------+----------+----+
|2006-01-01|        5|       19|     350|    2475|9.083333|12.483334|2006-01-01|null|
|2006-01-03|       -7|       -2|     344|    2475|8.883333|12.133333|2006-01-03|null|
|2006-01-04|       -5|      -13|     331|    2475|8.916667|    11.95|2006-01-04|null|
|2006-01-05|       -3|      -17|     321|    2475|    8.95|11.883333|2006-01-05|null|
|2006-01-06|       -4|      -32|     320|    2475|8.933333|11.633333|2006-01-06|null|
|2006-01-08|       -3|       -2|     346|    2475|    8.95|12.133333|2006-01-08|null|
|2006-01-09|        3|        0|     334|    2475|    9.05|12.166667|2006-01-09|null|
|2006-01-10|       -7|      -21|     334|    2475|8.883333|11.816667|2006-01-10|null|
|2006-01-11|        8|      -10|     321|    2475|9.13

In [61]:
# Ensure correct data types.
# Convert FL_DATE column to timestamp data type.
flights_df_convert = flights_df.withColumn("FL_DATE", col("FL_DATE").cast("timestamp"))
flights_df_convert.show()

+-------------------+---------+---------+--------+--------+---------+---------+----------+----+
|            FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|      Date|Time|
+-------------------+---------+---------+--------+--------+---------+---------+----------+----+
|2006-01-01 00:00:00|        5|       19|     350|    2475| 9.083333|12.483334|2006-01-01|null|
|2006-01-02 00:00:00|      167|      216|     343|    2475|11.783334|15.766666|2006-01-02|null|
|2006-01-03 00:00:00|       -7|       -2|     344|    2475| 8.883333|12.133333|2006-01-03|null|
|2006-01-04 00:00:00|       -5|      -13|     331|    2475| 8.916667|    11.95|2006-01-04|null|
|2006-01-05 00:00:00|       -3|      -17|     321|    2475|     8.95|11.883333|2006-01-05|null|
|2006-01-06 00:00:00|       -4|      -32|     320|    2475| 8.933333|11.633333|2006-01-06|null|
|2006-01-08 00:00:00|       -3|       -2|     346|    2475|     8.95|12.133333|2006-01-08|null|
|2006-01-09 00:00:00|        3|        0

In [62]:
# Renaming columns(e.g., rename Dep_Delay to Departure_Delay).
flights_df_rename = flights_df.withColumnRenamed("Dep_Delay", "Departure_Delay")
flights_df_rename.show()

+----------+---------------+---------+--------+--------+---------+---------+----------+----+
|   FL_DATE|Departure_Delay|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|      Date|Time|
+----------+---------------+---------+--------+--------+---------+---------+----------+----+
|2006-01-01|              5|       19|     350|    2475| 9.083333|12.483334|2006-01-01|null|
|2006-01-02|            167|      216|     343|    2475|11.783334|15.766666|2006-01-02|null|
|2006-01-03|             -7|       -2|     344|    2475| 8.883333|12.133333|2006-01-03|null|
|2006-01-04|             -5|      -13|     331|    2475| 8.916667|    11.95|2006-01-04|null|
|2006-01-05|             -3|      -17|     321|    2475|     8.95|11.883333|2006-01-05|null|
|2006-01-06|             -4|      -32|     320|    2475| 8.933333|11.633333|2006-01-06|null|
|2006-01-08|             -3|       -2|     346|    2475|     8.95|12.133333|2006-01-08|null|
|2006-01-09|              3|        0|     334|    2475|     9.05|12.1